Test

In [10]:
import numpy as np

assert np.array([1,2,3])@np.array([1,2,3]) == 14, 'Math is not mathing'

In [11]:
print("hallo wereld")

hallo wereld


In [12]:
from mesa import Agent
from mesa import Model

In [ ]:
#https://mesa.readthedocs.io/stable/tutorials/intro_tutorial.html

class FED_Agent(Agent):
    """Agent that models the FED (Central bank of US)"""

    def __init__(self, model):
        super().__init__(model)
        self.interest_rate = 4.75 #current

    def print_func(self):
        print(f"The interest rate = {str(self.interest_rate)}. ID = {str(self.unique_id)}.")

class Bank_Agent(Agent):
    """Agent that models the banks"""
    def __init__(self, model):
        super().__init__(model)
        self.profit = 0

class energy_sector(Agent):
    """Agent that models the energy sector"""

class Model1(Model):
    """Model for attempt 1."""

    def __init__(self, n, seed=None):
        super().__init__(seed=seed)
        self.num_agents = n

        FED_Agent.create_agents(model=self, n=1)
        Bank_Agent.create_agents(model=self, n = 1)

    def step(self):
        """Advance the model by one step."""
        self.agents.shuffle_do("print_func")

In [17]:
starter_model = Model1(10)
starter_model.step()

The interest rate = 4.75. ID = 1.
